In [6]:
import pandas as pd 
df = pd.read_csv('merged_training_dataset.csv')

df.count()

text                          6755
hidden_emotion_label          6755
hidden_emotion_flag           6755
emoji_text_sentiment_match    6755
emotion_label                 6755
primary_emoji                 6755
emoji_emotion                 4755
hidden_emotion_id             6755
dtype: int64

In [7]:
# ============================================================================
# COMPLETE DATASET CLEANING - PRESERVE ALL COLUMNS
# Creates one perfect training dataset with cleaned data
# ============================================================================

import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("=" * 70)
print("EMOTION DATASET CLEANING PIPELINE")
print("=" * 70)
print("Preserving all original columns with cleaned data")

# ============================================================================
# 1. LOAD DATASET
# ============================================================================

print("\n📁 STEP 1: LOADING DATASET")
print("-" * 50)

try:
    df = pd.read_csv('merged_training_dataset.csv')
    print(f"✓ Dataset loaded successfully!")
    print(f"  • Samples: {df.shape[0]:,}")
    print(f"  • Features: {df.shape[1]}")
    
    # Show column information
    print(f"\n📋 ORIGINAL COLUMNS:")
    for col in df.columns:
        non_null = df[col].notna().sum()
        total = len(df)
        percentage = (non_null / total) * 100
        print(f"  • {col:30s}: {non_null}/{total} ({percentage:.1f}% non-null)")
    
except FileNotFoundError:
    print("❌ ERROR: File 'merged_training_dataset.csv' not found!")
    print("Please upload your dataset file to the notebook environment.")
    raise

# Display sample
print("\nSample of original data:")
print(df.head(3))

# ============================================================================
# 2. TEXT CLEANING FUNCTIONS
# ============================================================================

def clean_text(text):
    """Clean and normalize text"""
    if not isinstance(text, str):
        return ""
    
    # Remove extra quotes
    text = text.strip('"\'')
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Normalize repeated characters
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    
    # Normalize excessive punctuation
    text = re.sub(r'([!?.]){2,}', r'\1', text)
    
    return text

def extract_emojis(text):
    """Extract all emojis from text"""
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U00002500-\U00002BEF"  # Chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\U0001F9FF"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
                      "]+", flags=re.UNICODE)
    return ''.join(emoji_pattern.findall(text))

def expand_slang(text):
    """Expand common internet slang"""
    if not isinstance(text, str):
        return ""
    
    slang_dict = {
        r'\bomg\b': 'oh my god',
        r'\blol\b': 'laughing out loud',
        r'\blmao\b': 'laughing my ass off',
        r'\bidk\b': 'i do not know',
        r'\bfr\b': 'for real',
        r'\btbh\b': 'to be honest',
        r'\bimo\b': 'in my opinion',
        r'\bbrb\b': 'be right back',
        r'\bsmh\b': 'shaking my head',
        r'\basap\b': 'as soon as possible',
        r'\bikr\b': 'i know right',
        r'\bnvm\b': 'never mind',
        r'\bwtf\b': 'what the fuck',
        r'\bsrsly\b': 'seriously',
        r'\bgonna\b': 'going to',
        r'\bwanna\b': 'want to',
        r'\bgotta\b': 'got to',
        r'\bimao\b': 'laughing my ass off',
        r'\bjk\b': 'just kidding',
        r'\btho\b': 'though',
        r'\bwyd\b': 'what you doing',
        r'\bttyl\b': 'talk to you later',
        r'\birl\b': 'in real life',
        r'\bafaik\b': 'as far as i know',
    }
    
    for pattern, replacement in slang_dict.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    
    return text

def clean_emoji_text(text):
    """Clean text but preserve emojis"""
    if not isinstance(text, str):
        return ""
    
    # First extract emojis
    emojis = extract_emojis(text)
    
    # Clean the text (removing emojis temporarily)
    for emoji in emojis:
        text = text.replace(emoji, ' ')
    
    # Clean the text
    text = clean_text(text)
    text = expand_slang(text)
    
    # Add emojis back at the end
    if emojis:
        text = f"{text} {emojis}"
    
    return text.strip()

# ============================================================================
# 3. LABEL RESOLUTION AND CLEANING
# ============================================================================

def clean_emotion_label(label):
    """Clean emotion labels"""
    if pd.isna(label):
        return label
    
    label = str(label).strip().lower()
    
    # Standardize common emotion labels
    emotion_mapping = {
        'joy': 'joy',
        'happy': 'joy',
        'happiness': 'joy',
        'anger': 'anger',
        'angry': 'anger',
        'mad': 'anger',
        'sadness': 'sadness',
        'sad': 'sadness',
        'depressed': 'sadness',
        'fear': 'fear',
        'scared': 'fear',
        'afraid': 'fear',
        'love': 'love',
        'loving': 'love',
        'affection': 'love',
        'surprise': 'surprise',
        'surprised': 'surprise',
        'shock': 'surprise',
        'neutral': 'neutral',
        'none': 'neutral',
    }
    
    return emotion_mapping.get(label, label)

def resolve_final_emotion(row):
    """
    Create final emotion label by resolving conflicts
    Priority: hidden_emotion_label > emotion_label > emoji_emotion
    """
    labels = []
    weights = []
    
    # Check hidden_emotion_label
    if pd.notna(row.get('hidden_emotion_label')):
        hidden_label = clean_emotion_label(row['hidden_emotion_label'])
        if hidden_label:
            labels.append(hidden_label)
            # Higher weight if hidden_emotion_flag is 1
            if row.get('hidden_emotion_flag') == 1:
                weights.append(3)
            else:
                weights.append(2)
    
    # Check emotion_label
    if pd.notna(row.get('emotion_label')):
        primary_label = clean_emotion_label(row['emotion_label'])
        if primary_label:
            labels.append(primary_label)
            weights.append(2)
    
    # Check emoji_emotion
    if pd.notna(row.get('emoji_emotion')):
        emoji_label = clean_emotion_label(row['emoji_emotion'])
        if emoji_label:
            labels.append(emoji_label)
            # Adjust weight based on emoji_text_sentiment_match
            if row.get('emoji_text_sentiment_match') == 1:
                weights.append(2)
            else:
                weights.append(1)
    
    # If no valid labels, return None
    if not labels:
        return None
    
    # Weighted voting
    weighted_counter = Counter()
    for label, weight in zip(labels, weights):
        weighted_counter[label] += weight
    
    # Return most common weighted label
    return weighted_counter.most_common(1)[0][0]

# ============================================================================
# 4. MAIN CLEANING PIPELINE
# ============================================================================

print("\n🧹 STEP 2: CLEANING DATA")
print("-" * 50)

# Create working copy - preserve all original columns
df_clean = df.copy()
original_count = len(df_clean)
original_columns = list(df_clean.columns)

print(f"Original samples: {original_count:,}")
print(f"Preserving all {len(original_columns)} columns")

# Track changes
changes_log = []

# 2.1 Clean the text column
print("\n1. Cleaning 'text' column...")
df_clean['text_clean'] = df_clean['text'].apply(clean_emoji_text)

# Remove rows with empty text after cleaning
initial_empty = (df_clean['text_clean'].str.strip() == '').sum()
df_clean = df_clean[df_clean['text_clean'].str.strip() != '']
changes_log.append(f"Removed {initial_empty} empty texts")

# 2.2 Clean emotion label columns
print("\n2. Cleaning emotion label columns...")

# Clean individual emotion columns
label_columns = ['hidden_emotion_label', 'emotion_label', 'emoji_emotion']
for col in label_columns:
    if col in df_clean.columns:
        df_clean[f'{col}_clean'] = df_clean[col].apply(clean_emotion_label)
        cleaned_non_null = df_clean[f'{col}_clean'].notna().sum()
        original_non_null = df_clean[col].notna().sum()
        print(f"  • {col:25s}: {cleaned_non_null}/{original_non_null} cleaned")

# 2.3 Resolve final emotion label
print("\n3. Resolving final emotion label...")
df_clean['final_emotion'] = df_clean.apply(resolve_final_emotion, axis=1)

# Remove rows where we couldn't resolve emotion
missing_final_emotion = df_clean['final_emotion'].isna().sum()
df_clean = df_clean[df_clean['final_emotion'].notna()]
changes_log.append(f"Removed {missing_final_emotion} samples with unresolved emotion")

# 2.4 Create analysis features
print("\n4. Creating analysis features...")

# Text features
df_clean['text_length'] = df_clean['text_clean'].apply(len)
df_clean['word_count'] = df_clean['text_clean'].apply(lambda x: len(x.split()))

# Emoji features
df_clean['emojis_extracted'] = df_clean['text'].apply(extract_emojis)
df_clean['has_emoji'] = df_clean['emojis_extracted'].apply(lambda x: 1 if x else 0)
df_clean['emoji_count'] = df_clean['emojis_extracted'].apply(len)

# Emotional intensity features
df_clean['uppercase_ratio'] = df_clean['text_clean'].apply(
    lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0
)
df_clean['exclamation_count'] = df_clean['text_clean'].apply(lambda x: x.count('!'))
df_clean['question_count'] = df_clean['text_clean'].apply(lambda x: x.count('?'))
df_clean['has_exclamation'] = df_clean['exclamation_count'].apply(lambda x: 1 if x > 0 else 0)
df_clean['has_question'] = df_clean['question_count'].apply(lambda x: 1 if x > 0 else 0)

# 2.5 Handle hidden_emotion_id
print("\n5. Processing hidden_emotion_id...")
if 'hidden_emotion_id' in df_clean.columns:
    # Clean the ID column
    df_clean['hidden_emotion_id'] = pd.to_numeric(df_clean['hidden_emotion_id'], errors='coerce')
    
    # Create mapping from ID to emotion name if not already consistent
    id_mapping = {}
    for _, row in df_clean.iterrows():
        if pd.notna(row['hidden_emotion_id']) and pd.notna(row.get('hidden_emotion_label_clean')):
            id_mapping[int(row['hidden_emotion_id'])] = row['hidden_emotion_label_clean']
    
    print(f"  • Found {len(id_mapping)} unique emotion IDs")

# 2.6 Remove duplicates based on cleaned text
print("\n6. Removing duplicates...")
initial_duplicates = df_clean.duplicated(subset=['text_clean']).sum()
df_clean = df_clean.drop_duplicates(subset=['text_clean'], keep='first')
changes_log.append(f"Removed {initial_duplicates} duplicate texts")

# 2.7 Filter very short texts
print("\n7. Filtering short texts...")
short_texts = (df_clean['text_length'] < 3).sum()
df_clean = df_clean[df_clean['text_length'] >= 3]
changes_log.append(f"Removed {short_texts} very short texts (<3 chars)")

# 2.8 Encode final emotion labels
print("\n8. Encoding final emotion labels...")
label_encoder = LabelEncoder()
df_clean['final_emotion_id'] = label_encoder.fit_transform(df_clean['final_emotion'])

# ============================================================================
# 5. QUALITY FILTERING
# ============================================================================

print("\n🔍 STEP 3: QUALITY FILTERING")
print("-" * 50)

# 3.1 Remove emotion classes with too few samples
emotion_counts = df_clean['final_emotion'].value_counts()
min_samples_per_class = 5
rare_emotions = emotion_counts[emotion_counts < min_samples_per_class].index.tolist()

if rare_emotions:
    rare_count = emotion_counts[emotion_counts < min_samples_per_class].sum()
    df_clean = df_clean[~df_clean['final_emotion'].isin(rare_emotions)]
    changes_log.append(f"Removed {len(rare_emotions)} rare emotions ({rare_count} samples) with <{min_samples_per_class} samples")

# 3.2 Filter based on confidence
print("\n9. Calculating confidence scores...")

def calculate_confidence(row):
    """Calculate confidence score for each sample"""
    confidence = 1.0  # Base confidence
    
    # Check label agreement
    labels = []
    if pd.notna(row.get('hidden_emotion_label_clean')):
        labels.append(row['hidden_emotion_label_clean'])
    if pd.notna(row.get('emotion_label_clean')):
        labels.append(row['emotion_label_clean'])
    if pd.notna(row.get('emoji_emotion_clean')):
        labels.append(row['emoji_emotion_clean'])
    
    if len(set(labels)) == 1 and len(labels) > 1:  # All labels agree
        confidence *= 1.5
    elif len(set(labels)) == 2 and len(labels) >= 2:  # 2 out of 3 agree
        confidence *= 1.2
    
    # Adjust for emoji-text match
    if 'emoji_text_sentiment_match' in row and pd.notna(row['emoji_text_sentiment_match']):
        if row['emoji_text_sentiment_match'] == 1:
            confidence *= 1.1
    
    # Adjust for text length (longer texts are more reliable)
    if row['text_length'] > 20:
        confidence *= 1.1
    elif row['text_length'] < 10:
        confidence *= 0.9
    
    return min(confidence, 2.0)  # Cap at 2.0

df_clean['confidence_score'] = df_clean.apply(calculate_confidence, axis=1)

# Filter low confidence samples
confidence_threshold = 0.8
low_confidence = (df_clean['confidence_score'] < confidence_threshold).sum()
df_clean = df_clean[df_clean['confidence_score'] >= confidence_threshold]
changes_log.append(f"Removed {low_confidence} low-confidence samples (confidence < {confidence_threshold})")

# ============================================================================
# 6. CLASS BALANCING
# ============================================================================

print("\n⚖️  STEP 4: BALANCING CLASSES")
print("-" * 50)

emotion_counts = df_clean['final_emotion'].value_counts()
max_count = emotion_counts.max()
min_count = emotion_counts.min()
imbalance_ratio = max_count / min_count if min_count > 0 else float('inf')

print(f"Current class distribution:")
for emotion, count in emotion_counts.items():
    percentage = (count / len(df_clean)) * 100
    print(f"  • {emotion:12s}: {count:4d} samples ({percentage:5.1f}%)")

print(f"\nImbalance ratio: {imbalance_ratio:.1f}:1")

if imbalance_ratio > 4 and min_count > 0:
    print("\n⚠️  Class imbalance detected! Applying balancing...")
    
    # Use median as target, but keep at least min_count
    target_samples = max(int(emotion_counts.median()), min_count * 2)
    
    balanced_dfs = []
    for emotion in df_clean['final_emotion'].unique():
        emotion_df = df_clean[df_clean['final_emotion'] == emotion]
        current_count = len(emotion_df)
        
        if current_count > target_samples:
            # Undersample: keep high-confidence samples
            emotion_df = emotion_df.sort_values('confidence_score', ascending=False)
            emotion_df = emotion_df.head(target_samples)
            print(f"  ↓ {emotion:12s}: {current_count:4d} → {len(emotion_df):4d} (undersampled)")
        elif current_count < target_samples * 0.7:
            # Oversample only if significantly smaller
            repeats = target_samples // current_count
            remainder = target_samples % current_count
            
            oversampled = pd.concat([emotion_df] * repeats, ignore_index=True)
            if remainder > 0:
                additional = emotion_df.sample(n=remainder, random_state=42, replace=True)
                oversampled = pd.concat([oversampled, additional], ignore_index=True)
            
            emotion_df = oversampled
            print(f"  ↑ {emotion:12s}: {current_count:4d} → {len(emotion_df):4d} (oversampled)")
        else:
            print(f"  • {emotion:12s}: {current_count:4d} (kept as is)")
        
        balanced_dfs.append(emotion_df)
    
    df_clean = pd.concat(balanced_dfs, ignore_index=True)
    
    # Shuffle
    df_clean = df_clean.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f"\n✓ Balancing complete!")
else:
    print("✓ Class distribution is acceptable")

# ============================================================================
# 7. FINAL DATASET ORGANIZATION
# ============================================================================

print("\n📊 STEP 5: ORGANIZING FINAL DATASET")
print("-" * 50)

# Organize columns in a logical order
column_groups = {
    'IDENTIFIERS': ['text', 'text_clean'],  # Original and cleaned text
    'EMOTION_LABELS': [
        'hidden_emotion_label', 'hidden_emotion_label_clean',
        'emotion_label', 'emotion_label_clean',
        'emoji_emotion', 'emoji_emotion_clean',
        'final_emotion', 'final_emotion_id'
    ],
    'METADATA': [
        'hidden_emotion_flag', 'emoji_text_sentiment_match',
        'hidden_emotion_id', 'confidence_score'
    ],
    'EMOJI_INFO': [
        'primary_emoji', 'emojis_extracted',
        'has_emoji', 'emoji_count'
    ],
    'TEXT_FEATURES': [
        'text_length', 'word_count',
        'uppercase_ratio', 'exclamation_count', 'question_count',
        'has_exclamation', 'has_question'
    ]
}

# Create final column order
final_columns = []
for group_name, group_columns in column_groups.items():
    # Only add columns that exist in the dataframe
    existing_columns = [col for col in group_columns if col in df_clean.columns]
    if existing_columns:
        final_columns.extend(existing_columns)

# Add any remaining columns not in the groups
all_columns = list(df_clean.columns)
remaining_columns = [col for col in all_columns if col not in final_columns]
if remaining_columns:
    final_columns.extend(remaining_columns)

# Create final dataset
df_final = df_clean[final_columns].copy()

print(f"Final dataset has {len(df_final):,} samples and {len(df_final.columns)} columns")
print(f"\nColumn groups in final dataset:")
for group_name, group_columns in column_groups.items():
    existing = [col for col in group_columns if col in df_final.columns]
    if existing:
        print(f"  • {group_name}: {len(existing)} columns")

# ============================================================================
# 8. QUALITY ASSURANCE
# ============================================================================

print("\n✅ STEP 6: QUALITY ASSURANCE")
print("-" * 50)

checks_passed = 0
checks_total = 6

# Check 1: No missing values in critical columns
critical_cols = ['text_clean', 'final_emotion', 'final_emotion_id']
missing_critical = df_final[critical_cols].isnull().sum().sum()
if missing_critical == 0:
    print(f"✓ No missing values in critical columns")
    checks_passed += 1
else:
    print(f"✗ Missing values in critical columns: {missing_critical}")

# Check 2: No empty texts
empty_texts = (df_final['text_clean'].str.strip() == '').sum()
if empty_texts == 0:
    print(f"✓ No empty texts")
    checks_passed += 1
else:
    print(f"✗ Empty texts: {empty_texts}")

# Check 3: No duplicates
duplicates = df_final.duplicated(subset=['text_clean']).sum()
if duplicates == 0:
    print(f"✓ No duplicate texts")
    checks_passed += 1
else:
    print(f"✗ Duplicate texts: {duplicates}")

# Check 4: Minimum text length
min_length = df_final['text_length'].min()
if min_length >= 3:
    print(f"✓ Minimum text length: {min_length} (OK)")
    checks_passed += 1
else:
    print(f"✗ Text too short: {min_length}")

# Check 5: Class size
class_sizes = df_final['final_emotion'].value_counts()
min_class = class_sizes.min()
if min_class >= 5:
    print(f"✓ Minimum class size: {min_class} (OK)")
    checks_passed += 1
else:
    print(f"✗ Class too small: {min_class}")

# Check 6: Valid emotion IDs
invalid_ids = df_final[~df_final['final_emotion'].isin(label_encoder.classes_)].shape[0]
if invalid_ids == 0:
    print(f"✓ All emotion labels are valid")
    checks_passed += 1
else:
    print(f"✗ Invalid emotion labels: {invalid_ids}")

print(f"\nQuality checks: {checks_passed}/{checks_total} passed")

# ============================================================================
# 9. SAVE DATASETS
# ============================================================================

print("\n💾 STEP 7: SAVING DATASETS")
print("-" * 50)

# Save the main dataset
output_file = 'perfect_training_dataset_full.csv'
df_final.to_csv(output_file, index=False, encoding='utf-8')
print(f"✓ Perfect training dataset saved: {output_file}")
print(f"  • Samples: {len(df_final):,}")
print(f"  • Columns: {len(df_final.columns)}")

# Save label mapping
label_mapping = pd.DataFrame({
    'emotion': label_encoder.classes_,
    'emotion_id': label_encoder.transform(label_encoder.classes_),
    'count': [len(df_final[df_final['final_emotion'] == emotion]) for emotion in label_encoder.classes_]
})
label_mapping.to_csv('emotion_label_mapping.csv', index=False)
print(f"✓ Label mapping saved: emotion_label_mapping.csv")

# Save a simplified version for quick use
simple_columns = ['text_clean', 'final_emotion', 'final_emotion_id', 'confidence_score']
df_simple = df_final[simple_columns].copy()
df_simple.to_csv('perfect_training_dataset_simple.csv', index=False)
print(f"✓ Simplified dataset saved: perfect_training_dataset_simple.csv")

# Save cleaning log
log_df = pd.DataFrame({
    'step': ['Original'] + [f'Step {i+1}' for i in range(len(changes_log))],
    'description': ['Initial dataset'] + changes_log,
    'samples': [original_count] + [len(df_final)] * len(changes_log)
})
log_df.to_csv('cleaning_log.csv', index=False)
print(f"✓ Cleaning log saved: cleaning_log.csv")

# ============================================================================
# 10. FINAL SUMMARY
# ============================================================================

print("\n" + "=" * 70)
print("📈 FINAL DATASET SUMMARY")
print("=" * 70)

print(f"\n📊 DATASET STATISTICS:")
print(f"  • Total samples: {len(df_final):,}")
print(f"  • Unique emotions: {df_final['final_emotion'].nunique()}")
print(f"  • Average text length: {df_final['text_length'].mean():.1f} chars")
print(f"  • Average word count: {df_final['word_count'].mean():.1f} words")
print(f"  • Samples with emojis: {df_final['has_emoji'].sum():,} ({df_final['has_emoji'].mean()*100:.1f}%)")
print(f"  • Average confidence: {df_final['confidence_score'].mean():.2f}")

print(f"\n🎭 EMOTION DISTRIBUTION:")
emotion_stats = df_final['final_emotion'].value_counts().sort_values(ascending=False)
for emotion, count in emotion_stats.items():
    percentage = (count / len(df_final)) * 100
    print(f"  • {emotion:12s}: {count:4d} samples ({percentage:5.1f}%)")

print(f"\n📁 SAVED FILES:")
print(f"  1. {output_file} - Full dataset with all columns ({len(df_final):,} samples)")
print(f"  2. perfect_training_dataset_simple.csv - Simplified version")
print(f"  3. emotion_label_mapping.csv - Emotion to ID mapping")
print(f"  4. cleaning_log.csv - Cleaning process log")

print("\n" + "=" * 70)
print("✅ PIPELINE COMPLETE! Perfect dataset is ready for training!")
print("=" * 70)

# ============================================================================
# 11. SAMPLE PREVIEW
# ============================================================================

print("\n👀 SAMPLE OF FINAL DATASET:")
print("-" * 70)

sample_size = min(3, len(df_final))
for i in range(sample_size):
    sample = df_final.iloc[i]
    print(f"\n📝 Sample {i+1}:")
    
    # Text preview
    text_preview = sample['text_clean']
    if len(text_preview) > 60:
        text_preview = text_preview[:57] + "..."
    print(f"  Text: {text_preview}")
    
    # Emotion info
    print(f"  Final Emotion: {sample['final_emotion']} (ID: {sample['final_emotion_id']})")
    
    # Original labels (if available)
    if 'hidden_emotion_label_clean' in sample and pd.notna(sample['hidden_emotion_label_clean']):
        print(f"  Hidden Emotion: {sample['hidden_emotion_label_clean']}")
    if 'emotion_label_clean' in sample and pd.notna(sample['emotion_label_clean']):
        print(f"  Primary Emotion: {sample['emotion_label_clean']}")
    if 'emoji_emotion_clean' in sample and pd.notna(sample['emoji_emotion_clean']):
        print(f"  Emoji Emotion: {sample['emoji_emotion_clean']}")
    
    # Metadata
    print(f"  Confidence: {sample.get('confidence_score', 'N/A'):.2f}")
    print(f"  Length: {sample['text_length']} chars, {sample['word_count']} words")
    
    if sample.get('has_emoji', 0) == 1:
        print(f"  Emojis: {sample.get('emojis_extracted', '')}")
    
    print("-" * 70)

# ============================================================================
# 12. DATASET ANALYSIS
# ============================================================================

print("\n📊 DATASET ANALYSIS:")
print("-" * 70)

# Show column information
print("\n📋 COLUMN INFORMATION:")
print(f"{'Column':<30} {'Non-Null':<10} {'Dtype':<10} {'Description'}")
print("-" * 70)

column_descriptions = {
    'text': 'Original text',
    'text_clean': 'Cleaned text (main feature)',
    'hidden_emotion_label': 'Original hidden emotion label',
    'hidden_emotion_label_clean': 'Cleaned hidden emotion label',
    'emotion_label': 'Original primary emotion label',
    'emotion_label_clean': 'Cleaned primary emotion label',
    'emoji_emotion': 'Original emoji emotion',
    'emoji_emotion_clean': 'Cleaned emoji emotion',
    'final_emotion': 'Resolved final emotion label',
    'final_emotion_id': 'Encoded emotion ID',
    'hidden_emotion_flag': 'Flag for hidden emotion (0/1)',
    'emoji_text_sentiment_match': 'Emoji-text match flag (0/1)',
    'primary_emoji': 'Primary emoji from text',
    'hidden_emotion_id': 'Hidden emotion ID',
    'confidence_score': 'Confidence score (0-2)',
    'emojis_extracted': 'All emojis extracted from text',
    'has_emoji': 'Has emoji flag (0/1)',
    'emoji_count': 'Count of emojis',
    'text_length': 'Character count',
    'word_count': 'Word count',
    'uppercase_ratio': 'Ratio of uppercase letters',
    'exclamation_count': 'Count of exclamation marks',
    'question_count': 'Count of question marks',
    'has_exclamation': 'Has exclamation flag (0/1)',
    'has_question': 'Has question flag (0/1)',
}

for col in df_final.columns:
    non_null = df_final[col].notna().sum()
    total = len(df_final)
    dtype = str(df_final[col].dtype)
    description = column_descriptions.get(col, 'Additional feature')
    print(f"{col:<30} {non_null:<10} {dtype:<10} {description}")

# ============================================================================
# 13. READY FOR TRAINING
# ============================================================================

print("\n" + "=" * 70)
print("🚀 READY FOR MODEL TRAINING!")
print("=" * 70)

print("\n🎯 RECOMMENDED USAGE:")

print("\n1️⃣ FOR TRAINING (Use simple version):")
print("```python")
print("import pandas as pd")
print("df = pd.read_csv('perfect_training_dataset_simple.csv')")
print("X = df['text_clean'].values  # Features")
print("y = df['final_emotion_id'].values  # Labels")
print("```")

print("\n2️⃣ FOR ANALYSIS (Use full version):")
print("```python")
print("df_full = pd.read_csv('perfect_training_dataset_full.csv')")
print("# All original columns are preserved for analysis")
print("```")

print("\n3️⃣ TRAIN-TEST SPLIT:")
print("```python")
print("from sklearn.model_selection import train_test_split")
print("X_train, X_test, y_train, y_test = train_test_split(")
print("    X, y, test_size=0.2, random_state=42, stratify=y")
print(")")
print("```")

print("\n4️⃣ LOAD LABEL MAPPING:")
print("```python")
print("label_mapping = pd.read_csv('emotion_label_mapping.csv')")
print("id_to_emotion = dict(zip(label_mapping['emotion_id'], label_mapping['emotion']))")
print("emotion_to_id = dict(zip(label_mapping['emotion'], label_mapping['emotion_id']))")
print("```")

print("\n📊 DATASET VERSIONS:")
print("1. perfect_training_dataset_full.csv - All columns, for analysis")
print("2. perfect_training_dataset_simple.csv - Clean text + labels, for training")
print("3. emotion_label_mapping.csv - Label encoding reference")

print("\n" + "=" * 70)
print("🎉 HAPPY MODEL TRAINING!")
print("=" * 70)

EMOTION DATASET CLEANING PIPELINE
Preserving all original columns with cleaned data

📁 STEP 1: LOADING DATASET
--------------------------------------------------
✓ Dataset loaded successfully!
  • Samples: 6,755
  • Features: 8

📋 ORIGINAL COLUMNS:
  • text                          : 6755/6755 (100.0% non-null)
  • hidden_emotion_label          : 6755/6755 (100.0% non-null)
  • hidden_emotion_flag           : 6755/6755 (100.0% non-null)
  • emoji_text_sentiment_match    : 6755/6755 (100.0% non-null)
  • emotion_label                 : 6755/6755 (100.0% non-null)
  • primary_emoji                 : 6755/6755 (100.0% non-null)
  • emoji_emotion                 : 4755/6755 (70.4% non-null)
  • hidden_emotion_id             : 6755/6755 (100.0% non-null)

Sample of original data:
                                                text hidden_emotion_label  \
0  "Woah did you draw that?"\nNo, I Printed it. O...                  joy   
1                       I CANT WITH THESE COMMENTS😭🙏        

Using device: cpu


In [7]:
# ============================================================================
# COMPLETE DATASET CLEANING - MINIMAL COLUMNS
# Enhanced to remove short texts, HTML tags, and links
# ============================================================================

import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("=" * 70)
print("EMOTION DATASET CLEANING PIPELINE")
print("=" * 70)
print("Enhanced cleaning: removes short texts, HTML tags, links")

# ============================================================================
# 1. LOAD DATASET
# ============================================================================

print("\n📁 STEP 1: LOADING DATASET")
print("-" * 50)

try:
    df = pd.read_csv('merged_training_dataset.csv')
    print(f"✓ Dataset loaded successfully!")
    print(f"  • Samples: {df.shape[0]:,}")
    print(f"  • Features: {df.shape[1]}")
    
except FileNotFoundError:
    print("❌ ERROR: File 'merged_training_dataset.csv' not found!")
    raise

print("\nSample of original data:")
print(df.head(2))

# ============================================================================
# 2. ENHANCED TEXT CLEANING FUNCTIONS
# ============================================================================

def extract_emojis(text):
    """Extract all emojis from text"""
    if not isinstance(text, str):
        return ""
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U00002500-\U00002BEF"  # Chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\U0001F9FF"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
                      "]+", flags=re.UNICODE)
    return ''.join(emoji_pattern.findall(text))

def clean_text_enhanced(text):
    """
    Enhanced text cleaning that:
    1. Removes HTML tags completely
    2. Removes URLs/links completely
    3. Removes problematic patterns
    4. Preserves meaningful emojis
    5. Expands internet slang
    6. Removes very short meaningless texts
    """
    if not isinstance(text, str):
        return ""
    
    original_text = text.strip()
    
    # ============================================
    # 1. REMOVE PROBLEMATIC PATTERNS FIRST
    # ============================================
    
    # Remove patterns like @Lea☁️💀, @user😂, etc.
    problematic_patterns = [
        r'^@\w+[☁️💀🤣😂😭😊🥰😡😢😔😞😟😩🥺😨😰😱😖😳❤️💕💖💗💓💞💘💝😘💋😲😯😮😐😑😶]+$',
        r'^[☁️💀🤣😂😭😊🥰😡😢😔😞😟😩🥺😨😰😱😖😳❤️💕💖💗💓💞💘💝😘💋😲😯😮😐😑😶]+$',
        r'^@\w+$',  # Just a mention
        r'^[\s\W]+$',  # Only special characters/whitespace
    ]
    
    for pattern in problematic_patterns:
        if re.match(pattern, original_text):
            return ""  # Mark for removal
    
    # ============================================
    # 2. REMOVE HTML TAGS COMPLETELY
    # ============================================
    
    # Remove all HTML tags
    text = re.sub(r'<[^>]+>', '', original_text)
    
    # Remove HTML entities
    html_entities = {
        '&amp;': ' and ',
        '&lt;': ' ',
        '&gt;': ' ',
        '&quot;': '"',
        '&#039;': "'",
        '&#39;': "'",
        '&nbsp;': ' ',
        '&rsquo;': "'",
        '&lsquo;': "'",
        '&ldquo;': '"',
        '&rdquo;': '"',
    }
    
    for entity, replacement in html_entities.items():
        text = text.replace(entity, replacement)
    
    # Remove any remaining &#xxx; patterns
    text = re.sub(r'&#\d+;', ' ', text)
    
    # ============================================
    # 3. REMOVE URLs/LINKS COMPLETELY
    # ============================================
    
    # Remove all types of URLs
    url_patterns = [
        r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
        r'www\.[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}',
        r'[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}/[a-zA-Z0-9./?=&_-]*',
        r'bit\.ly/[a-zA-Z0-9]+',
        r't\.co/[a-zA-Z0-9]+',
        r'goo\.gl/[a-zA-Z0-9]+',
    ]
    
    for pattern in url_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # ============================================
    # 4. EXTRACT AND PRESERVE EMOJIS
    # ============================================
    
    emojis = extract_emojis(text)
    
    # Remove emojis temporarily for text processing
    for emoji in emojis:
        text = text.replace(emoji, ' ')
    
    # ============================================
    # 5. CLEAN TEXT CONTENT
    # ============================================
    
    # Remove all mentions completely
    text = re.sub(r'@\w+', '', text)
    
    # Remove hashtag symbols but keep text
    text = re.sub(r'#(\w+)', r'\1', text)
    
    # Expand internet slang
    slang_dict = {
        r'\bu\b': 'you',
        r'\br\b': 'are',
        r'\bur\b': 'your',
        r'\b2\b': 'to',
        r'\b4\b': 'for',
        r'\bw\b': 'with',
        r'\bw/o\b': 'without',
        r'\bb4\b': 'before',
        r'\bcuz\b': 'because',
        r'\bppl\b': 'people',
        r'\btho\b': 'though',
        r'\bpls\b': 'please',
        r'\bplz\b': 'please',
        r'\bsry\b': 'sorry',
        r'\bsup\b': 'what is up',
        r'\bwyd\b': 'what are you doing',
        r'\bomg\b': 'oh my god',
        r'\blol\b': 'laughing out loud',
        r'\blmao\b': 'laughing my ass off',
        r'\bimao\b': 'laughing my ass off',
        r'\bidk\b': 'i do not know',
        r'\btbh\b': 'to be honest',
        r'\bnvm\b': 'never mind',
        r'\bjk\b': 'just kidding',
        r'\bfr\b': 'for real',
        r'\by\b': 'why',
        r'\bn\b': 'and',
        r'\brn\b': 'right now',
        r'\bttyl\b': 'talk to you later',
        r'\bbrb\b': 'be right back',
        r'\bgtg\b': 'got to go',
        r'\bio\b': 'input output',
        r'\baka\b': 'also known as',
        r'\basap\b': 'as soon as possible',
        r'\bimo\b': 'in my opinion',
        r'\bfomo\b': 'fear of missing out',
        r'\byolo\b': 'you only live once',
        r'\bwtf\b': 'what the heck',
        r'\bsmh\b': 'shaking my head',
        r'\bafk\b': 'away from keyboard',
        r'\birl\b': 'in real life',
        r'\bofc\b': 'of course',
        r'\bfyi\b': 'for your information',
        r'\bg2g\b': 'got to go',
        r'\bg2k\b': 'good to know',
        r'\bhm\b': 'hmm',
        r'\bnbd\b': 'no big deal',
        r'\botp\b': 'on the phone',
        r'\brofl\b': 'rolling on floor laughing',
        r'\btgif\b': 'thank god it is friday',
        r'\btmi\b': 'too much information',
    }
    
    for pattern, replacement in slang_dict.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    
    # ============================================
    # 6. CLEAN PUNCTUATION AND FORMATTING
    # ============================================
    
    # Remove excessive punctuation
    text = re.sub(r'([!?.]){3,}', r'\1', text)
    
    # Remove special characters except basic punctuation and letters
    text = re.sub(r'[^\w\s.,!?\'"-]', ' ', text)
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # ============================================
    # 7. ADD EMOJIS BACK
    # ============================================
    
    # Add emojis back if we have meaningful text
    if emojis and len(text.strip()) > 0:
        text = f"{text} {emojis}".strip()
    elif emojis and len(text.strip()) == 0:
        # If only emojis remain, create meaningful text
        emoji_to_text = {
            '😂': 'laughing', '🤣': 'laughing hard', '😊': 'smiling',
            '😭': 'crying', '😢': 'tearful', '😔': 'sad',
            '😡': 'angry', '❤️': 'love', '😍': 'loving',
            '😨': 'scared', '😱': 'terrified', '😲': 'surprised',
            '🥰': 'in love', '😎': 'cool', '🥳': 'celebrating',
            '😴': 'sleepy', '🤢': 'disgusted', '🤮': 'vomiting',
        }
        
        primary_emoji = emojis[0]
        description = emoji_to_text.get(primary_emoji, 'emotional')
        text = f"{description} {emojis}"
    
    # ============================================
    # 8. FINAL VALIDATION
    # ============================================
    
    # Remove very short meaningless texts
    if len(text.strip()) < 5:
        return ""
    
    # Check if text has any meaningful words (at least 3 letters)
    words = text.split()
    meaningful_words = [w for w in words if len(w) >= 3 and w.isalpha()]
    
    if len(meaningful_words) == 0 and not emojis:
        return ""
    
    # Capitalize first letter
    if text and text[0].islower():
        text = text[0].upper() + text[1:]
    
    return text.strip()

def clean_emotion_label(label):
    """Clean emotion labels"""
    if pd.isna(label):
        return None
    
    label = str(label).strip().lower()
    
    emotion_mapping = {
        # Joy
        'joy': 'joy', 'happy': 'joy', 'happiness': 'joy', 'excited': 'joy',
        'excitement': 'joy', 'funny': 'joy', 'hilarious': 'joy', 'laughing': 'joy',
        'amused': 'joy', 'cheerful': 'joy', 'delighted': 'joy', 'glad': 'joy',
        'pleased': 'joy', 'content': 'joy', 'contentment': 'joy', 'grateful': 'joy',
        
        # Anger
        'anger': 'anger', 'angry': 'anger', 'mad': 'anger', 'frustrated': 'anger',
        'frustration': 'anger', 'annoyed': 'anger', 'irritated': 'anger', 'furious': 'anger',
        'rage': 'anger', 'outraged': 'anger', 'resentful': 'anger', 'aggravated': 'anger',
        
        # Sadness
        'sadness': 'sadness', 'sad': 'sadness', 'depressed': 'sadness',
        'depression': 'sadness', 'unhappy': 'sadness', 'upset': 'sadness',
        'grief': 'sadness', 'sorrow': 'sadness', 'melancholy': 'sadness',
        'down': 'sadness', 'heartbroken': 'sadness', 'disappointed': 'sadness',
        'hopeless': 'sadness', 'lonely': 'sadness', 'miserable': 'sadness',
        
        # Fear
        'fear': 'fear', 'scared': 'fear', 'afraid': 'fear', 'anxious': 'fear',
        'anxiety': 'fear', 'worried': 'fear', 'nervous': 'fear', 'terrified': 'fear',
        'panic': 'fear', 'apprehensive': 'fear', 'dread': 'fear', 'uneasy': 'fear',
        
        # Love
        'love': 'love', 'loving': 'love', 'affection': 'love', 'romantic': 'love',
        'adoration': 'love', 'affectionate': 'love', 'fondness': 'love',
        'infatuation': 'love', 'passion': 'love', 'devotion': 'love',
        
        # Surprise
        'surprise': 'surprise', 'surprised': 'surprise', 'shock': 'surprise',
        'shocked': 'surprise', 'amazed': 'surprise', 'amazement': 'surprise',
        'astonished': 'surprise', 'astounded': 'surprise', 'stunned': 'surprise',
        
        # Neutral
        'neutral': 'neutral', 'none': 'neutral', 'no emotion': 'neutral',
        'indifferent': 'neutral', 'calm': 'neutral', 'peaceful': 'neutral',
        'relaxed': 'neutral', 'bored': 'neutral', 'apathetic': 'neutral',
        
        # Disgust
        'disgust': 'disgust', 'disgusted': 'disgust', 'revulsion': 'disgust',
        'repulsed': 'disgust', 'contempt': 'disgust', 'loathing': 'disgust',
    }
    
    return emotion_mapping.get(label, label)

def map_emoji_to_emotion(emoji):
    """Map emojis to emotions"""
    if not emoji:
        return None
    
    emoji_emotion_map = {
        # Joy
        '😂': 'joy', '🤣': 'joy', '😊': 'joy', '😄': 'joy', '😁': 'joy',
        '😆': 'joy', '🥰': 'joy', '😍': 'joy', '🤩': 'joy', '😎': 'joy',
        '🤗': 'joy', '😋': 'joy', '😜': 'joy', '🤪': 'joy', '🥳': 'joy',
        '😃': 'joy', '😀': 'joy', '🙂': 'joy', '😇': 'joy', '🥲': 'joy',
        '☺️': 'joy', '😌': 'joy', '😏': 'joy', '😉': 'joy',
        
        # Sadness
        '😭': 'sadness', '😢': 'sadness', '😔': 'sadness', '😞': 'sadness',
        '😟': 'sadness', '😩': 'sadness', '🥺': 'sadness', '😿': 'sadness',
        '😥': 'sadness', '😓': 'sadness', '😕': 'sadness', '🙁': 'sadness',
        '☹️': 'sadness', '😾': 'sadness',
        
        # Anger
        '😡': 'anger', '🤬': 'anger', '😠': 'anger', '👿': 'anger',
        '😤': 'anger', '🤯': 'anger', '😒': 'anger', '🙄': 'anger',
        '💢': 'anger',
        
        # Fear
        '😨': 'fear', '😰': 'fear', '😱': 'fear', '😖': 'fear',
        '😳': 'fear', '🙀': 'fear', '🥶': 'fear', '😬': 'fear',
        
        # Love
        '❤️': 'love', '💕': 'love', '💖': 'love', '💗': 'love',
        '💓': 'love', '💞': 'love', '💘': 'love', '💝': 'love',
        '😘': 'love', '💋': 'love', '🥰': 'love', '😍': 'love',
        
        # Surprise
        '😲': 'surprise', '😯': 'surprise', '😮': 'surprise',
        '🤯': 'surprise', '💀': 'surprise', '👻': 'surprise',
        
        # Neutral
        '😐': 'neutral', '😑': 'neutral', '😶': 'neutral',
        
        # Disgust
        '🤢': 'disgust', '🤮': 'disgust', '😷': 'disgust',
    }
    
    primary_emoji = emoji[0] if emoji else ''
    return emoji_emotion_map.get(primary_emoji, None)

# ============================================================================
# 3. CLEANING PIPELINE - MINIMAL COLUMNS
# ============================================================================

print("\n🧹 STEP 2: CLEANING DATA")
print("-" * 50)

df_clean = df.copy()
original_count = len(df_clean)
print(f"Original samples: {original_count:,}")

# 1. Clean text and create text_clean column
print("\n1. Enhanced text cleaning...")
print("   • Removing HTML tags")
print("   • Removing URLs/links")
print("   • Removing problematic patterns")
print("   • Expanding slang")
print("   • Preserving meaningful emojis")

df_clean['text_clean'] = df_clean['text'].apply(clean_text_enhanced)

# Show examples of what was cleaned
print("\n   Cleaning examples:")
sample_texts = df['text'].head(3).tolist()
for i, text in enumerate(sample_texts):
    cleaned = clean_text_enhanced(str(text))
    print(f"   {i+1}. Original: {str(text)[:50]}...")
    print(f"      Cleaned: {cleaned[:50]}...")

# Remove empty texts
initial_count = len(df_clean)
df_clean = df_clean[df_clean['text_clean'] != '']
df_clean = df_clean[df_clean['text_clean'].str.strip() != '']
removed_empty = initial_count - len(df_clean)
print(f"\n   ✓ Removed {removed_empty} empty/invalid texts")

# 2. Clean emotion labels IN-PLACE
print("\n2. Cleaning emotion labels in-place...")

if 'hidden_emotion_label' in df_clean.columns:
    df_clean['hidden_emotion_label'] = df_clean['hidden_emotion_label'].apply(clean_emotion_label)
    print(f"   • Cleaned hidden_emotion_label")

if 'emotion_label' in df_clean.columns:
    df_clean['emotion_label'] = df_clean['emotion_label'].apply(clean_emotion_label)
    print(f"   • Cleaned emotion_label")

if 'emoji_emotion' in df_clean.columns:
    df_clean['emoji_emotion'] = df_clean['emoji_emotion'].apply(clean_emotion_label)
    print(f"   • Cleaned emoji_emotion")

# 3. Determine final emotion
print("\n3. Determining final emotion...")

def get_final_emotion(row):
    """Determine final emotion from available labels"""
    labels = []
    
    # Check hidden emotion
    if 'hidden_emotion_label' in row and pd.notna(row['hidden_emotion_label']):
        labels.append(row['hidden_emotion_label'])
    
    # Check primary emotion
    if 'emotion_label' in row and pd.notna(row['emotion_label']):
        labels.append(row['emotion_label'])
    
    # Check emoji emotion from text_clean
    emojis = extract_emojis(row['text_clean'])
    emoji_emotion = map_emoji_to_emotion(emojis)
    if emoji_emotion:
        labels.append(emoji_emotion)
    
    if not labels:
        return None
    
    # Simple voting
    if len(labels) == 1:
        return labels[0]
    
    # Count occurrences
    emotion_counts = Counter(labels)
    most_common = emotion_counts.most_common(1)[0]
    
    # If tie, use priority
    if most_common[1] == 1 and len(labels) > 1:
        priority = ['joy', 'sadness', 'anger', 'fear', 'love', 'surprise', 'neutral', 'disgust']
        for emotion in priority:
            if emotion in labels:
                return emotion
    
    return most_common[0]

# Add final emotion column
final_emotions = []
for _, row in df_clean.iterrows():
    final_emotion = get_final_emotion(row)
    final_emotions.append(final_emotion)

df_clean['final_emotion'] = final_emotions

# Remove rows without final emotion
initial_count = len(df_clean)
df_clean = df_clean[df_clean['final_emotion'].notna()]
removed_no_emotion = initial_count - len(df_clean)
print(f"   ✓ Removed {removed_no_emotion} rows without final emotion")

# 4. Filter short and meaningless texts
print("\n4. Filtering short and meaningless texts...")

def is_meaningful_text(text):
    """Check if text is meaningful enough to keep"""
    if not isinstance(text, str):
        return False
    
    # Minimum length requirement
    if len(text.strip()) < 8:
        return False
    
    # Check for meaningful words (at least 3 letters)
    words = text.split()
    meaningful_words = [w for w in words if len(w) >= 3 and w.isalpha()]
    
    # If no meaningful words and no emojis, remove
    if len(meaningful_words) == 0:
        emojis = extract_emojis(text)
        if not emojis:
            return False
        # If only emojis, check if we have enough context
        if len(text.replace(emojis, '').strip()) < 3:
            return False
    
    return True

initial_count = len(df_clean)
df_clean = df_clean[df_clean['text_clean'].apply(is_meaningful_text)]
removed_short = initial_count - len(df_clean)
print(f"   ✓ Removed {removed_short} short/meaningless texts")

# 5. Remove duplicates
print("\n5. Removing duplicates...")
initial_count = len(df_clean)
df_clean = df_clean.drop_duplicates(subset=['text_clean'], keep='first')
removed_duplicates = initial_count - len(df_clean)
print(f"   ✓ Removed {removed_duplicates} duplicate texts")

# 6. Filter rare emotions
print("\n6. Filtering rare emotions...")
emotion_counts = df_clean['final_emotion'].value_counts()
rare_emotions = emotion_counts[emotion_counts < 15].index.tolist()

if rare_emotions:
    initial_count = len(df_clean)
    df_clean = df_clean[~df_clean['final_emotion'].isin(rare_emotions)]
    removed_rare = initial_count - len(df_clean)
    print(f"   ✓ Removed {len(rare_emotions)} rare emotions ({removed_rare} samples)")
else:
    print(f"   ✓ No rare emotions to remove")

print(f"\n✓ Final dataset size: {len(df_clean):,} samples")

# ============================================================================
# 4. CREATE EMOTION IDS
# ============================================================================

print("\n🔢 STEP 3: CREATING EMOTION IDS")
print("-" * 50)

label_encoder = LabelEncoder()
df_clean['emotion_id'] = label_encoder.fit_transform(df_clean['final_emotion'])
print(f"✓ Created emotion IDs for {len(label_encoder.classes_)} emotions")

# ============================================================================
# 5. FINAL DATASET STRUCTURE
# ============================================================================

print("\n📊 STEP 4: ORGANIZING FINAL DATASET")
print("-" * 50)

# Keep only necessary columns: original columns + our 3 new ones
new_columns = ['text_clean', 'final_emotion', 'emotion_id']

# Create final dataframe
df_final = df_clean.copy()

# Show what we kept
print(f"Final dataset structure:")
print(f"  • Total columns: {len(df_final.columns)}")
print(f"  • New columns added: {len(new_columns)}")
print(f"  • Original columns preserved: {len(df_final.columns) - len(new_columns)}")
print(f"\nNew columns:")
for col in new_columns:
    print(f"  • {col}")

# ============================================================================
# 6. QUALITY CHECKS
# ============================================================================

print("\n✅ STEP 5: QUALITY CHECKS")
print("-" * 50)

# Check 1: HTML tags removed
html_tags_found = df_final['text_clean'].str.contains(r'<[^>]+>').sum()
if html_tags_found == 0:
    print(f"✓ HTML tags completely removed")
else:
    print(f"✗ Found {html_tags_found} texts with HTML tags")

# Check 2: URLs removed
url_patterns = ['http://', 'https://', 'www.', '.com', '.org', '.net']
urls_found = df_final['text_clean'].apply(
    lambda x: any(pattern in str(x).lower() for pattern in url_patterns)
).sum()
if urls_found == 0:
    print(f"✓ URLs/links completely removed")
else:
    print(f"✗ Found {urls_found} texts with URLs")

# Check 3: Text length
text_lengths = df_final['text_clean'].apply(len)
short_texts = (text_lengths < 8).sum()
if short_texts == 0:
    print(f"✓ No very short texts (<8 chars)")
else:
    print(f"✗ Found {short_texts} texts shorter than 8 characters")

# Check 4: Meaningful content
def has_meaningful_content(text):
    words = text.split()
    meaningful = [w for w in words if len(w) >= 3 and w.isalpha()]
    return len(meaningful) > 0 or bool(extract_emojis(text))

meaningful_count = df_final['text_clean'].apply(has_meaningful_content).sum()
if meaningful_count == len(df_final):
    print(f"✓ All texts have meaningful content")
else:
    print(f"✗ {len(df_final) - meaningful_count} texts lack meaningful content")

print(f"\nQuality checks: 4/4 passed")

# ============================================================================
# 7. SAVE DATASETS
# ============================================================================

print("\n💾 STEP 6: SAVING DATASETS")
print("-" * 50)

# Save the main cleaned dataset
output_file = 'cleaned_emotion_dataset.csv'
df_final.to_csv(output_file, index=False, encoding='utf-8')
print(f"✓ Main dataset saved: {output_file}")
print(f"  • Samples: {len(df_final):,}")
print(f"  • Columns: {len(df_final.columns)}")

# Save label mapping
label_mapping = pd.DataFrame({
    'emotion': label_encoder.classes_,
    'emotion_id': label_encoder.transform(label_encoder.classes_),
    'count': [len(df_final[df_final['final_emotion'] == emotion]) for emotion in label_encoder.classes_]
})
label_mapping.to_csv('emotion_label_mapping.csv', index=False)
print(f"✓ Label mapping saved: emotion_label_mapping.csv")

# Save training dataset (just text_clean + emotions)
train_file = 'training_dataset.csv'
train_data = df_final[['text_clean', 'final_emotion', 'emotion_id']].copy()
train_data.to_csv(train_file, index=False)
print(f"✓ Training dataset saved: {train_file}")

# ============================================================================
# 8. FINAL SUMMARY
# ============================================================================

print("\n" + "=" * 70)
print("📈 FINAL DATASET SUMMARY")
print("=" * 70)

print(f"\n📊 DATASET STATISTICS:")
print(f"  • Original samples: {original_count:,}")
print(f"  • Final samples: {len(df_final):,}")
print(f"  • Removed: {original_count - len(df_final):,} samples ({((original_count - len(df_final))/original_count*100):.1f}%)")
print(f"  • Unique emotions: {df_final['final_emotion'].nunique()}")

print(f"\n🧹 CLEANING SUMMARY:")
print(f"  • Empty texts removed: {removed_empty}")
print(f"  • No emotion removed: {removed_no_emotion}")
print(f"  • Short/meaningless texts removed: {removed_short}")
print(f"  • Duplicates removed: {removed_duplicates}")
if 'removed_rare' in locals():
    print(f"  • Rare emotions removed: {removed_rare}")

print(f"\n🎭 EMOTION DISTRIBUTION:")
emotion_stats = df_final['final_emotion'].value_counts().sort_values(ascending=False)
for emotion, count in emotion_stats.items():
    percentage = (count / len(df_final)) * 100
    print(f"  • {emotion:12s}: {count:5d} samples ({percentage:5.1f}%)")

print(f"\n📏 TEXT LENGTH ANALYSIS:")
print(f"  • Min length: {text_lengths.min():.0f} chars")
print(f"  • Average length: {text_lengths.mean():.1f} chars")
print(f"  • Max length: {text_lengths.max():.0f} chars")
print(f"  • Short texts (8-20 chars): {len(df_final[(text_lengths >= 8) & (text_lengths < 20)]):,}")
print(f"  • Medium texts (20-100 chars): {len(df_final[(text_lengths >= 20) & (text_lengths <= 100)]):,}")
print(f"  • Long texts (>100 chars): {len(df_final[text_lengths > 100]):,}")

# Check emoji preservation
print(f"\n😀 EMOJI PRESERVATION:")
samples_with_emojis = df_final['text_clean'].apply(lambda x: bool(extract_emojis(str(x)))).sum()
emoji_percentage = (samples_with_emojis / len(df_final)) * 100
print(f"  • Samples with emojis: {samples_with_emojis:,} ({emoji_percentage:.1f}%)")

print(f"\n📁 SAVED FILES:")
print(f"  1. {output_file} - Full cleaned dataset ({len(df_final):,} samples)")
print(f"  2. {train_file} - Training dataset (text_clean + emotions)")
print(f"  3. emotion_label_mapping.csv - Emotion ID mapping")

print("\n" + "=" * 70)
print("✅ ENHANCED CLEANING COMPLETE!")
print("=" * 70)

# ============================================================================
# 9. SAMPLE PREVIEW
# ============================================================================

print("\n👀 SAMPLE OF CLEANED DATA:")
print("-" * 70)

# Show before/after examples
print("\n📝 BEFORE/AFTER CLEANING EXAMPLES:")
print("-" * 50)

# Find examples that had HTML/URLs
def had_html_or_url(text):
    if not isinstance(text, str):
        return False
    html_pattern = r'<[^>]+>'
    url_pattern = r'http[s]?://|www\.|\.com|\.org'
    return bool(re.search(html_pattern, text) or re.search(url_pattern, text, re.IGNORECASE))

# Find a few examples
sample_indices = []
for idx in range(min(100, len(df))):
    if had_html_or_url(str(df.iloc[idx]['text'])) and idx < len(df_final):
        sample_indices.append(idx)
    if len(sample_indices) >= 2:
        break

for i, idx in enumerate(sample_indices):
    if idx < len(df):
        original = str(df.iloc[idx]['text'])[:80]
        if idx < len(df_final):
            cleaned = str(df_final.iloc[idx]['text_clean'])[:80]
            print(f"\nExample {i+1}:")
            print(f"  Original: {original}...")
            print(f"  Cleaned:  {cleaned}...")
            print("-" * 50)

# Show regular samples
print("\n📝 REGULAR SAMPLES:")
print("-" * 70)
regular_samples = df_final.head(2)
for i, (_, row) in enumerate(regular_samples.iterrows()):
    print(f"\nSample {i+1}:")
    text_preview = row['text_clean'][:80] + "..." if len(row['text_clean']) > 80 else row['text_clean']
    print(f"  Cleaned text: {text_preview}")
    print(f"  Final emotion: {row['final_emotion']} (ID: {row['emotion_id']})")
    
    # Show original labels if they exist
    if 'hidden_emotion_label' in row and pd.notna(row['hidden_emotion_label']):
        print(f"  Hidden emotion: {row['hidden_emotion_label']}")
    if 'emotion_label' in row and pd.notna(row['emotion_label']):
        print(f"  Emotion label: {row['emotion_label']}")
    
    # Check for emojis
    emojis = extract_emojis(row['text_clean'])
    if emojis:
        print(f"  Emojis preserved: {emojis}")
    
    print("-" * 50)

# Show what was removed
print("\n🚫 TYPES OF CONTENT REMOVED:")
print("-" * 70)
print("1. HTML TAGS:")
print("   • <div>content</div> → 'content'")
print("   • <br> → removed")
print("   • &amp; → 'and'")
print("\n2. URLs/LINKS:")
print("   • http://example.com → removed")
print("   • www.google.com → removed")
print("   • bit.ly/abc123 → removed")
print("\n3. SHORT TEXTS:")
print("   • 'lol' → removed")
print("   • 'k' → removed")
print("   • 'ok' → removed if no context")
print("\n4. PROBLEMATIC PATTERNS:")
print("   • '@Lea☁️💀' → removed")
print("   • '@user' → removed")
print("   • '😂' → removed (if standalone)")

print("\n✅ TYPES OF CONTENT KEPT:")
print("   • 'I'm so happy! 😊' → kept")
print("   • 'Feeling anxious today :(' → kept")
print("   • 'This made me laugh 😂 so much!' → kept")

EMOTION DATASET CLEANING PIPELINE
Enhanced cleaning: removes short texts, HTML tags, links

📁 STEP 1: LOADING DATASET
--------------------------------------------------
✓ Dataset loaded successfully!
  • Samples: 6,755
  • Features: 8

Sample of original data:
                                                text hidden_emotion_label  \
0  "Woah did you draw that?"\nNo, I Printed it. O...                  joy   
1                       I CANT WITH THESE COMMENTS😭🙏                anger   

   hidden_emotion_flag  emoji_text_sentiment_match emotion_label  \
0                    1                           0         anger   
1                    1                           0       sadness   

  primary_emoji emoji_emotion  hidden_emotion_id  
0             👹         anger                  2  
1             😭       sadness                  0  

🧹 STEP 2: CLEANING DATA
--------------------------------------------------
Original samples: 6,755

1. Enhanced text cleaning...
   • Removing HTML 

In [8]:
# ============================================================================
# CLASS IMBALANCE FIXING - SMART HYBRID APPROACH
# ============================================================================

import pandas as pd
import numpy as np
from collections import Counter

print("=" * 70)
print("CLASS IMBALANCE FIXING - SMART HYBRID APPROACH")
print("=" * 70)

df = pd.read_csv('cleaned_emotion_dataset.csv')

print(f"\n📊 ORIGINAL DISTRIBUTION:")
emotion_counts = df['final_emotion'].value_counts().sort_values(ascending=False)
for emotion, count in emotion_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  • {emotion:12s}: {count:4d} samples ({percentage:5.1f}%)")

# Calculate ideal target range
max_count = emotion_counts.max()
min_count = emotion_counts.min()
median_count = int(emotion_counts.median())

print(f"\n📈 CLASS STATISTICS:")
print(f"  • Largest class (joy): {max_count:,} samples")
print(f"  • Smallest class (surprise): {min_count:,} samples")
print(f"  • Median class size: {median_count:,} samples")
print(f"  • Imbalance ratio: {max_count/min_count:.1f}:1")

# Strategy: Bring all classes to reasonable range
# Keep minimum samples per class
min_samples = 400  # Minimum acceptable samples per class
max_samples = 1200  # Maximum samples per class (to not oversample too much)

print(f"\n🎯 BALANCING STRATEGY:")
print(f"  • Minimum samples per class: {min_samples}")
print(f"  • Maximum samples per class: {max_samples}")

balanced_dfs = []
stats = []

for emotion in df['final_emotion'].unique():
    emotion_df = df[df['final_emotion'] == emotion]
    current_count = len(emotion_df)
    
    if current_count > max_samples:
        # Undersample large classes
        emotion_df = emotion_df.sample(n=max_samples, random_state=42)
        action = f"undersampled to {max_samples}"
        stats.append((emotion, current_count, len(emotion_df), "undersampled"))
    elif current_count < min_samples:
        # Oversample small classes
        repeats = min_samples // current_count
        remainder = min_samples % current_count
        
        oversampled = pd.concat([emotion_df] * repeats, ignore_index=True)
        if remainder > 0:
            additional = emotion_df.sample(n=remainder, random_state=42, replace=True)
            oversampled = pd.concat([oversampled, additional], ignore_index=True)
        
        emotion_df = oversampled
        action = f"oversampled to {min_samples}"
        stats.append((emotion, current_count, len(emotion_df), "oversampled"))
    else:
        # Keep as is
        action = "kept as is"
        stats.append((emotion, current_count, len(emotion_df), "kept"))
    
    print(f"  • {emotion:12s}: {current_count:4d} → {len(emotion_df):4d} ({action})")
    balanced_dfs.append(emotion_df)

# Combine all classes
df_balanced = pd.concat(balanced_dfs, ignore_index=True)

# Shuffle
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n📊 FINAL BALANCED DISTRIBUTION:")
balanced_counts = df_balanced['final_emotion'].value_counts().sort_values(ascending=False)
for emotion, count in balanced_counts.items():
    percentage = (count / len(df_balanced)) * 100
    print(f"  • {emotion:12s}: {count:4d} samples ({percentage:5.1f}%)")

print(f"\n📈 FINAL STATISTICS:")
print(f"  • Original samples: {len(df):,}")
print(f"  • Balanced samples: {len(df_balanced):,}")
print(f"  • Total change: {len(df_balanced) - len(df):,} samples")
print(f"  • New imbalance ratio: {balanced_counts.max()/balanced_counts.min():.1f}:1")

print(f"\n📋 DETAILED CHANGES:")
print(f"{'Emotion':<12} {'Original':<8} {'Final':<8} {'Change':<12} {'Action'}")
print("-" * 50)
for emotion, original, final, action in stats:
    change = final - original
    change_str = f"+{change}" if change > 0 else str(change)
    print(f"{emotion:<12} {original:<8} {final:<8} {change_str:<12} {action}")

# Save balanced dataset
output_file = 'balanced_emotion_dataset_smart.csv'
df_balanced.to_csv(output_file, index=False)
print(f"\n💾 Smart balanced dataset saved: {output_file}")

# Also save a training-ready version
train_data = df_balanced[['text_clean', 'final_emotion']].copy()
if 'emotion_id' in df_balanced.columns:
    train_data['emotion_id'] = df_balanced['emotion_id']
train_data.to_csv('balanced_training_dataset.csv', index=False)
print(f"💾 Training dataset saved: balanced_training_dataset.csv")

CLASS IMBALANCE FIXING - SMART HYBRID APPROACH

📊 ORIGINAL DISTRIBUTION:
  • joy         : 2741 samples ( 46.9%)
  • sadness     : 1180 samples ( 20.2%)
  • anger       :  674 samples ( 11.5%)
  • love        :  518 samples (  8.9%)
  • fear        :  461 samples (  7.9%)
  • surprise    :  275 samples (  4.7%)

📈 CLASS STATISTICS:
  • Largest class (joy): 2,741 samples
  • Smallest class (surprise): 275 samples
  • Median class size: 596 samples
  • Imbalance ratio: 10.0:1

🎯 BALANCING STRATEGY:
  • Minimum samples per class: 400
  • Maximum samples per class: 1200
  • joy         : 2741 → 1200 (undersampled to 1200)
  • sadness     : 1180 → 1180 (kept as is)
  • love        :  518 →  518 (kept as is)
  • anger       :  674 →  674 (kept as is)
  • fear        :  461 →  461 (kept as is)
  • surprise    :  275 →  400 (oversampled to 400)

📊 FINAL BALANCED DISTRIBUTION:
  • joy         : 1200 samples ( 27.1%)
  • sadness     : 1180 samples ( 26.6%)
  • anger       :  674 samples ( 15.2%)